Install Libraries

In [ ]:
'''
Dependency Explanations:

sys:            Provides access to Python interpreter variables and functions, used here
                to check Python version

libreoffice:    Office suite used to convert DOCX files to PDF format for processing

poppler-utils:  PDF rendering library that powers the pdf2image package for PDF to
                image conversion

tesseract-ocr:  Open-source OCR engine that extracts text from images

torch:          PyTorch deep learning framework that provides tensor computation and
                neural networks

torchvision:    Computer vision library that works with PyTorch for image processing tasks

pdf2image:      Converts PDF documents to images using poppler under the hood

opencv-python:  Computer vision library for image processing operations

layoutparser:   Library for document layout analysis and segmentation

pytesseract:    Python wrapper for Google's Tesseract OCR engine

cython:         Language that makes writing C extensions for Python easier, required
                for detectron2

pycocotools:    Tools for working with the COCO dataset format, used by detectron2

fvcore:         Core libraries for computer vision research, used by detectron2

detectron2:     Facebook AI Research's object detection and segmentation framework
                used by layoutparser
'''


# Check Python version
import sys
print(f"Python version: {sys.version}")

# Install system dependencies
!apt-get update -qq
!apt-get install -y libreoffice poppler-utils tesseract-ocr -qq  # For DOCX to PDF, pdf2image, and OCR

# Clean up existing torch-related packages to avoid conflicts
#!pip uninstall torch torchaudio torchvision fastai layoutparser detectron2 fvcore -y --quiet

# Install latest Python dependencies
!pip install torch torchvision pdf2image opencv-python layoutparser pytesseract --quiet

# Install prerequisites for detectron2
!pip install cython pycocotools fvcore --quiet

# Install detectron2 from source (compatible with torch 2.6.0+cu124)
#!pip install 'git+https://github.com/facebookresearch/detectron2.git'

!pip install python-docx
!pip install --upgrade pillow
!sudo apt-get install tesseract-ocr-eng
#!pip install "detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.5#egg=detectron2"
!pip install -U 'git+https://github.com/facebookresearch/detectron2.git@ff53992b1985b63bd3262b5a36167098e3dada02'
!pip install "layoutparser[ocr]"

!pip install sentence-transformers faiss-cpu  fastapi uvicorn pyngrok
!pip install faiss-gpu

Python version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
  Cloning https://github.com/facebookresearch/detectron2.git (to revision ff53992b1985b63bd3262b5a36167098e3dada02) to /tmp/pip-req-build-87sj94_c
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-87sj94_c
  Running command git rev-parse -q --verify 'sha^ff53992b1985b63bd3262b5a36167098e3dada02'
  Running command git fetch -q https://github.com/facebookresearch/detectron2.git ff53992b1985b63bd3262b5a36167098e3dada02
  Running co

Download Model, Convert word->pdf->png, extract and classify content, store metadata and index

In [ ]:
import torch
import torchvision
import layoutparser as lp
import detectron2
import os
import subprocess
from datetime import datetime
from pathlib import Path
from pdf2image import convert_from_path
import cv2
import pytesseract
import numpy as np
from PIL import Image
from docx import Document
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
import re
from sentence_transformers import SentenceTransformer
import faiss
import psycopg2
from psycopg2.extras import execute_values
from google.colab import userdata
import shutil

# Configuration flag
USE_POSTGRES = True

# Neon PostgreSQL connection details
DB_CONFIG = {
    "dbname": userdata.get("DB_NAME"),
    "user": userdata.get("DB_USER"),
    "password": userdata.get("DB_PASSWORD"),
    "host": userdata.get("DB_HOST"),
    "port": userdata.get("DB_PORT"),  # Default PostgreSQL port
    "sslmode": "require"  # Neon requires SSL
}

# Verify installations
print(f"Torch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"LayoutParser version: {lp.__version__}")
print(f"Detectron2 version: {detectron2.__version__}")

# Device selection
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Mount Google Drive only in Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    print("Not running in Colab, skipping drive mount")

# Set up directories
BASE_DIR = "/content/drive/My Drive/lifesciences" if 'drive' in globals() else "./lifesciences"
SOURCE_DIR = Path(BASE_DIR) / "training_documents"
PDF_STAGING_DIR = Path(BASE_DIR) / "staging/pdf"
IMG_STAGING_DIR = Path(BASE_DIR) / "staging/images"
ANNOTATED_DIR = Path(BASE_DIR) / "staging/annotated"
FAISS_INDEX_PATH = Path(BASE_DIR) / "faiss_index.index"

for directory in [SOURCE_DIR, PDF_STAGING_DIR, IMG_STAGING_DIR, ANNOTATED_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

Image.MAX_IMAGE_PIXELS = 100000000

# Global variables with lazy initialization
id_seq = 1
sentence_model = None
faiss_index = None
model = None
ocr_agent = None
db_conn = None
db_cursor = None
run_number = 1
run_date_time = datetime.now().isoformat()
logged_in_user = "admin"

COLORS = {
    "title": (255, 0, 0), "text": (0, 255, 0), "list": (0, 0, 255),
    "table": (255, 255, 0), "figure": (255, 0, 255)
}

def lazy_load_models():
    """Load models with device awareness and FAISS GPU fallback"""
    global sentence_model, faiss_index, model, ocr_agent
    backup_dir = "/content/drive/My Drive/lifesciences/models"
    backup_model_path = os.path.join(backup_dir, 'model_final.pth')
    if sentence_model is None:
        sentence_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
        dimension = 384
        faiss_index = faiss.IndexFlatL2(dimension)
        if device.type == "cuda":
            try:
                res = faiss.StandardGpuResources()
                faiss_index = faiss.index_cpu_to_gpu(res, 0, faiss_index)
                print("FAISS GPU support enabled")
            except AttributeError:
                print("FAISS GPU support not available. Falling back to CPU.")
    if model is None:
        MODEL_CONFIG = "lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config"
        model_path = '/root/.torch/iopath_cache/s/57zjbwv6gh3srry/model_final.pth'
        try:
            model = lp.Detectron2LayoutModel(
                config_path=MODEL_CONFIG,
                extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.2],
                label_map={0: "Text", 1: "Title", 2: "List", 3: "Table", 4: "Figure"},
                device=str(device)
            )
        except Exception as e:
            print(f"Model loading error: {e}")
            try:
                os.rename('/root/.torch/iopath_cache/s/57zjbwv6gh3srry/model_final.pth?dl=1', model_path)
                model = lp.Detectron2LayoutModel(
                    config_path=MODEL_CONFIG,
                    model_path=model_path,
                    extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.2],
                    label_map={0: "Text", 1: "Title", 2: "List", 3: "Table", 4: "Figure"},
                    device=str(device)
                )
                os.makedirs(backup_dir, exist_ok=True)
                if os.path.exists(model_path) and not os.path.exists(backup_model_path):
                    shutil.copy(model_path, backup_model_path)
                    print(f"Backed up model to {backup_model_path}")
            except Exception as e2:
                print(f"Failed to load model after path fix: {e2}")
    if ocr_agent is None:
        ocr_agent = lp.TesseractAgent(languages='eng', config='--psm 6 --oem 1')

def setup_db_connection():
    """Initialize PostgreSQL connection to Neon and start batch run"""
    global db_conn, db_cursor, run_number
    if db_conn is None:
        try:
            db_conn = psycopg2.connect(**DB_CONFIG)
            db_cursor = db_conn.cursor()
            print("Connected to Neon PostgreSQL database")

            # Get the maximum run_number and increment it by 1
            db_cursor.execute("SELECT COALESCE(MAX(run_number), 0) FROM batch_run")
            max_run_number = db_cursor.fetchone()[0]
            run_number = max_run_number + 1

            # Insert new batch run with the calculated run_number and initial status 'RUNNING'
            insert_batch_query = """
                INSERT INTO batch_run (run_number, run_date_time, run_by_user, run_final_status)
                VALUES (%s, %s, %s, 'RUNNING')
            """
            db_cursor.execute(insert_batch_query, (run_number, run_date_time, logged_in_user))
            db_conn.commit()
            print(f"Started new batch run: {run_number}")

        except Exception as e:
            print(f"Failed to connect to Neon PostgreSQL: {e}")
            raise

def update_run_status(status):
    """Update the run_final_status for the current batch run"""
    global db_conn, db_cursor, run_number
    if db_conn is not None:
        try:
            update_query = """
                UPDATE batch_run
                SET run_final_status = %s
                WHERE run_number = %s
            """
            db_cursor.execute(update_query, (status, run_number))
            db_conn.commit()
            print(f"Updated batch run {run_number} status to {status}")
        except Exception as e:
            print(f"Failed to update run status: {e}")
            db_conn.rollback()

def add_table_border(table):
    """Optimized table border addition"""
    tbl = table._tbl
    tblBorders = OxmlElement('w:tblBorders')
    for border_name in ['top', 'left', 'bottom', 'right', 'insideH', 'insideV']:
        border = OxmlElement(f'w:{border_name}')
        border.set(qn('w:val'), 'single')
        border.set(qn('w:sz'), '12')
        tblBorders.append(border)
    tbl.append(tblBorders)

def adjust_docx_titles_and_spacing(docx_path, temp_docx_path):
    """Memory-efficient DOCX processing"""
    doc = Document(docx_path)
    for para in doc.paragraphs:
        if (para.style and "Heading" in para.style.name) or \
           (para.runs and all(run.bold for run in para.runs if run.text.strip())):
            text = para.text.strip()
            if text:
                para.clear()
                run = para.add_run(text.upper())
                run.bold = run.underline = True
    for table in doc.tables:
        add_table_border(table)
    doc.save(temp_docx_path)
    return temp_docx_path

def convert_docx_to_pdf(docx_path, output_dir):
    """Optimized DOCX to PDF conversion"""
    docx_path = Path(docx_path)
    output_dir = Path(output_dir)
    final_pdf = output_dir / f"{docx_path.stem}.pdf"
    temp_docx = output_dir / f"{docx_path.stem}_temp.docx"

    adjusted_docx = adjust_docx_titles_and_spacing(docx_path, temp_docx)
    try:
        subprocess.run([
            'libreoffice', '--headless', '--convert-to', 'pdf',
            str(adjusted_docx), '--outdir', str(output_dir)
        ], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        temp_pdf = output_dir / f"{temp_docx.stem}.pdf"
        if temp_pdf.exists():
            temp_pdf.rename(final_pdf)
        temp_docx.unlink(missing_ok=True)
        return str(final_pdf)
    except subprocess.CalledProcessError as e:
        print(f"Conversion error: {e}")
        return None

def convert_pdf_to_png(pdf_path, output_dir):
    """Memory-efficient PDF to PNG conversion with save verification"""
    pdf_name = Path(pdf_path).stem
    try:
        images = convert_from_path(pdf_path, dpi=300, thread_count=1)
        image_paths = []
        for i, img in enumerate(images):
            output_path = os.path.join(output_dir, f"{pdf_name}_page_{i + 1}.png")
            img.save(output_path, "PNG", quality=85, optimize=True)
            if os.path.exists(output_path):
                print(f"Saved image: {output_path}")
            else:
                print(f"Failed to save image: {output_path}")
            image_paths.append(output_path)
            img.close()
        del images
        return image_paths
    except Exception as e:
        print(f"PDF conversion error for {pdf_path}: {e}")
        return []

def is_overlapping(rect1, rect2):
    """Optimized overlap check"""
    return not (rect1[2] <= rect2[0] or rect2[2] <= rect1[0] or
                rect1[3] <= rect2[1] or rect2[3] <= rect1[1])

def append_skipped_blocks_to_db(skipped_blocks):
    """Write skipped blocks to the skipped_block_items table with truncation"""
    if not skipped_blocks:
        return

    setup_db_connection()
    try:
        MAX_CONTENT_LENGTH = 1000
        values = [
            (
                sb["run_number"],
                sb["parent_block_type"],
                sb["skipped_block_type"],
                sb["section_name"][:255] if sb["section_name"] else None,
                sb["parent_block_content"][:MAX_CONTENT_LENGTH-3] + "..." if len(sb["parent_block_content"]) > MAX_CONTENT_LENGTH else sb["parent_block_content"],
                sb["skipped_block_content"][:MAX_CONTENT_LENGTH-3] + "..." if len(sb["skipped_block_content"]) > MAX_CONTENT_LENGTH else sb["skipped_block_content"],
                sb["parent_block_coordinates"],
                sb["skipped_block_coordinates"]
            )
            for sb in skipped_blocks
        ]
        query = """
            INSERT INTO skipped_block_items (
                run_number, parent_block_type, skipped_block_type, section_name,
                parent_block_content, skipped_block_content,
                parent_block_coordinates, skipped_block_coordinates
            ) VALUES %s
        """
        execute_values(db_cursor, query, values)
        db_conn.commit()
        print(f"Inserted {len(skipped_blocks)} skipped blocks into skipped_block_items")
    except Exception as e:
        print(f"Skipped blocks database error: {e}")
        db_conn.rollback()

def process_layout(image_path, original_file_path):
    """Memory-efficient layout processing with skipped block tracking including content"""
    lazy_load_models()
    global id_seq

    image = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_COLOR)
    if image is None:
        raise ValueError(f"Failed to load image: {image_path}")

    layout = model.detect(image)
    sorted_layout = sorted(layout, key=lambda x: x.coordinates[1])
    results = []
    skipped_blocks = []
    list_table_regions = [(b.type.lower(), list(map(int, b.coordinates)))
                         for b in sorted_layout if b.type.lower() in ["list", "table"]]
    current_section = "No Title"

    list_table_contents = {}
    for block_type, coords in list_table_regions:
        cropped = image[coords[1]:coords[3], coords[0]:coords[2]]
        content = ocr_agent.detect(cropped) or ""
        if block_type == "list":
            content = re.sub(r'[^a-zA-Z0-9\s]', '', content)
        elif block_type == "table":
            content = '\n'.join('-'.join(col.strip() for col in row.split() if col.strip())
                               for row in content.split('\n') if row.strip()) or "Table Content"
        list_table_contents[tuple(coords)] = content.strip()
        del cropped

    block_number = 1

    for block in sorted_layout:
        coords = list(map(int, block.coordinates))
        block_type = block.type.lower()

        overlapping_region = next((r for r in list_table_regions if is_overlapping(coords, r[1])), None)
        if block_type in ["title", "text"] and overlapping_region:
            parent_type, parent_coords = overlapping_region
            cropped = image[coords[1]:coords[3], coords[0]:coords[2]]
            skipped_content = ocr_agent.detect(cropped) or ""
            if block_type == "title":
                skipped_content = skipped_content or "Untitled"

            print(f"Skipping {block_type} inside {parent_type} at coordinates: {coords}")
            skipped_blocks.append({
                "run_number": run_number,
                "parent_block_type": parent_type,
                "skipped_block_type": block_type,
                "section_name": current_section,
                "parent_block_content": list_table_contents[tuple(parent_coords)],
                "skipped_block_content": skipped_content.strip(),
                "parent_block_coordinates": parent_coords,
                "skipped_block_coordinates": coords
            })
            del cropped
            continue

        cropped = image[coords[1]:coords[3], coords[0]:coords[2]]
        if block_type in ["list", "table"]:
            content = list_table_contents[tuple(coords)]
        elif block_type == "title":
            content = ocr_agent.detect(cropped) or "Untitled"
            current_section = content.lower()
        else:
            content = ocr_agent.detect(cropped) or ""

        result = {
            "id": id_seq,
            "urs": Path(image_path).stem,
            "section": current_section,
            "type": block_type,
            "content": content.strip(),
            "coordinates": coords,
            "created_at": datetime.now().isoformat(),
            "source_file": str(original_file_path),
            "run_number": run_number,
            "block_number": block_number,
        }
        results.append(result)
        id_seq += 1
        block_number += 1

        x1, y1, x2, y2 = coords
        color = COLORS.get(block_type, (255, 255, 255))
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, f"{block_type} #{block_number-1}", (x1, y1-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        del cropped

    annotated_path = os.path.join(ANNOTATED_DIR, Path(image_path).stem + "_annotated.png")
    success = cv2.imwrite(annotated_path, image, [cv2.IMWRITE_PNG_COMPRESSION, 9])
    if success:
        print(f"Saved annotated image: {annotated_path}")
    else:
        print(f"Failed to save annotated image: {annotated_path}")

    del image

    append_skipped_blocks_to_db(skipped_blocks)

    return results

def append_to_db(data):
    """Write directly to PostgreSQL table with block_number"""
    if not data:
        return

    setup_db_connection()
    try:
        values = [
            (d["id"], d["urs"], d["section"], d["type"], d["content"],
             d["coordinates"][0], d["coordinates"][1], d["coordinates"][2], d["coordinates"][3],
             d["created_at"], d["source_file"], d["run_number"], d["block_number"])
            for d in data
        ]
        query = """
            INSERT INTO document_metadata (
                id, urs, section, type, content, coord_x1, coord_y1, coord_x2, coord_y2,
                created_at, source_file, run_number, block_number
            ) VALUES %s
            ON CONFLICT (id) DO UPDATE SET
                urs = EXCLUDED.urs,
                section = EXCLUDED.section,
                type = EXCLUDED.type,
                content = EXCLUDED.content,
                coord_x1 = EXCLUDED.coord_x1,
                coord_y1 = EXCLUDED.coord_y1,
                coord_x2 = EXCLUDED.coord_x2,
                coord_y2 = EXCLUDED.coord_y2,
                created_at = EXCLUDED.created_at,
                source_file = EXCLUDED.source_file,
                run_number = EXCLUDED.run_number,
                block_number = EXCLUDED.block_number
        """
        execute_values(db_cursor, query, values)
        db_conn.commit()
        print(f"Inserted/Updated {len(data)} records into PostgreSQL")
    except Exception as e:
        print(f"Database error: {e}")
        db_conn.rollback()

def process_file(file_path):
    """Process file and retain PNGs in IMG_STAGING_DIR"""
    file_ext = Path(file_path).suffix.lower()
    pdf_path = convert_docx_to_pdf(file_path, PDF_STAGING_DIR) if file_ext == '.docx' else file_path
    if not pdf_path or not os.path.exists(pdf_path):
        print(f"Skipping {file_path}: PDF conversion failed or file not found")
        return

    png_paths = convert_pdf_to_png(pdf_path, IMG_STAGING_DIR)
    results = []
    for png_path in png_paths:
        try:
            results.extend(process_layout(png_path, file_path))
            print(f"Processed and retained: {png_path}")
        except Exception as e:
            print(f"Layout error for {png_path}: {e}")
    append_to_db(results)
    return results

def build_faiss_index():
    """Memory-efficient FAISS index building"""
    lazy_load_models()
    setup_db_connection()
    try:
        db_cursor.execute("SELECT type, urs, section, content FROM document_metadata WHERE content IS NOT NULL")
        metadata = [{"type": row[0], "urs": row[1], "section": row[2], "content": row[3]} for row in db_cursor.fetchall()]
    except Exception as e:
        print(f"Error fetching from PostgreSQL: {e}")
        return

    if not metadata:
        return

    batch_size = 1000
    for i in range(0, len(metadata), batch_size):
        batch = metadata[i:i + batch_size]
        texts = [f"{e['type']} {e.get('urs', '')} {e.get('section', '')} {e.get('content', '')}"
                for e in batch if e.get("content", "").strip()]
        if texts:
            embeddings = sentence_model.encode(texts, batch_size=min(batch_size, len(texts)),
                                            convert_to_numpy=True, show_progress_bar=False)
            faiss_index.add(embeddings)
    faiss.write_index(faiss_index, str(FAISS_INDEX_PATH))
    print(f"FAISS index built and saved to {FAISS_INDEX_PATH}")

def process_and_index():
    """Main processing loop"""
    try:
        for filename in sorted(os.listdir(SOURCE_DIR)):
            file_path = os.path.join(SOURCE_DIR, filename)
            print(f"file_path: {file_path}")
            if filename.lower().endswith(('.docx', '.pdf')):
                process_file(file_path)
        build_faiss_index()
        update_run_status('SUCCESS')
    except Exception as e:
        print(f"Processing failed: {e}")
        update_run_status('FAILED')
        raise

def cleanup():
    """Close database connection and set TERMINATED status if needed"""
    global db_conn, db_cursor
    if db_conn is not None:
        try:
            db_cursor.execute("SELECT run_final_status FROM batch_run WHERE run_number = %s", (run_number,))
            current_status = db_cursor.fetchone()[0]
            if current_status == 'RUNNING':
                update_run_status('TERMINATED')
        except Exception as e:
            print(f"Error during cleanup status check: {e}")
        finally:
            db_cursor.close()
            db_conn.close()
            print("Database connection closed")

if __name__ == "__main__":
    try:
        subprocess.run(['nvidia-smi'], capture_output=True, text=True)  # Fixed subprocess syntax
        print("nvidia-smi output captured")
    except Exception as e:
        print(f"nvidia-smi not found or error: {e}")
    try:
        process_and_index()
    except KeyboardInterrupt:
        print("Process interrupted by user")
        update_run_status('TERMINATED')
    except Exception as e:
        print(f"Unexpected error: {e}")
        update_run_status('FAILED')
    finally:
        cleanup()

Torch version: 2.6.0+cu124
Torchvision version: 0.21.0+cu124
LayoutParser version: 0.3.4
Detectron2 version: 0.6
Using device: cpu
Mounted at /content/drive
nvidia-smi not found or error: [Errno 2] No such file or directory: 'nvidia-smi'
file_path: /content/drive/My Drive/lifesciences/training_documents/Pharma_URS_Enhanced.docx
Saved image: /content/drive/My Drive/lifesciences/staging/images/Pharma_URS_Enhanced_page_1.png
Saved image: /content/drive/My Drive/lifesciences/staging/images/Pharma_URS_Enhanced_page_2.png


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.yaml?dl=1: 8.19kB [00:01, 7.16kB/s]
model_final.pth?dl=1: 856MB [00:14, 57.5MB/s]                           


Model loading error: Checkpoint /root/.torch/iopath_cache/s/57zjbwv6gh3srry/model_final.pth not found!
Backed up model to /content/drive/My Drive/lifesciences/models/model_final.pth


/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Skipping text inside list at coordinates: [497, 2505, 905, 2553]
Saved annotated image: /content/drive/My Drive/lifesciences/staging/annotated/Pharma_URS_Enhanced_page_1_annotated.png
Connected to Neon PostgreSQL database
Started new batch run: 1
Inserted 1 skipped blocks into skipped_block_items
Processed and retained: /content/drive/My Drive/lifesciences/staging/images/Pharma_URS_Enhanced_page_1.png
Saved annotated image: /content/drive/My Drive/lifesciences/staging/annotated/Pharma_URS_Enhanced_page_2_annotated.png
Processed and retained: /content/drive/My Drive/lifesciences/staging/images/Pharma_URS_Enhanced_page_2.png
Inserted/Updated 29 records into PostgreSQL
file_path: /content/drive/My Drive/lifesciences/training_documents/ProtonGlow_Test_URS.docx
Saved image: /content/drive/My Drive/lifesciences/staging/images/ProtonGlow_Test_URS_page_1.png
Saved image: /content/drive/My Drive/lifesciences/staging/images/ProtonGlow_Test_URS_page_2.png
Saved image: /content/drive/My Drive/life

Query FAISS Index

In [ ]:
import faiss
from fastapi import FastAPI
import os
import psycopg2
from psycopg2.extras import execute_values
from sentence_transformers import SentenceTransformer
from google.colab import userdata

# FAISS index path
FAISS_INDEX_PATH = "/content/drive/My Drive/lifesciences/faiss_index.index"

# PostgreSQL connection details (same as previous code)
DB_CONFIG = {
    "dbname": userdata.get("DB_NAME"),
    "user": userdata.get("DB_USER"),
    "password": userdata.get("DB_PASSWORD"),
    "host": userdata.get("DB_HOST"),
    "port": userdata.get("DB_PORT"),
}

# Initialize SentenceTransformer
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')
print("Loaded SentenceTransformer: all-MiniLM-L6-v2")

# Database connection (will be initialized lazily)
db_conn = None
db_cursor = None

def setup_db_connection():
    """Initialize PostgreSQL connection"""
    global db_conn, db_cursor
    if db_conn is None:
        try:
            db_conn = psycopg2.connect(**DB_CONFIG)
            db_cursor = db_conn.cursor()
            print("Connected to PostgreSQL database for search")
        except Exception as e:
            print(f"Failed to connect to PostgreSQL: {e}")
            raise

def get_latest_run_metadata():
    """Fetch metadata from the latest run in document_metadata"""
    setup_db_connection()
    try:
        # Get the latest run_number from batch_run where status is 'SUCCESS'
        db_cursor.execute("""
            SELECT run_number
            FROM batch_run
            WHERE run_final_status = 'SUCCESS'
            ORDER BY run_number DESC
            LIMIT 1
        """)
        latest_run = db_cursor.fetchone()
        if not latest_run:
            print("No successful runs found in batch_run.")
            return []

        latest_run_number = latest_run[0]
        print(f"Fetching metadata for latest successful run: {latest_run_number}")

        # Fetch metadata from document_metadata for the latest run
        db_cursor.execute("""
            SELECT type, urs, section, content
            FROM document_metadata
            WHERE run_number = %s AND content IS NOT NULL
        """, (latest_run_number,))
        metadata = [
            {"type": row[0], "urs": row[1], "section": row[2], "content": row[3]}
            for row in db_cursor.fetchall()
        ]
        print(f"Retrieved {len(metadata)} metadata entries from run {latest_run_number}")
        return metadata
    except Exception as e:
        print(f"Error fetching latest run metadata: {e}")
        return []

def query_faiss_index(urs_name, section_name, content_type, top_k=10):
    """Query FAISS index with urs_name, section_name, and content_type, return matching database entries"""
    if not os.path.exists(FAISS_INDEX_PATH):
        print("FAISS index file not found at:", FAISS_INDEX_PATH)
        return []

    # Load FAISS index
    faiss_index = faiss.read_index(FAISS_INDEX_PATH)
    print("Loaded FAISS index from:", FAISS_INDEX_PATH)

    # Fetch metadata from the latest run in the database
    metadata_entries = get_latest_run_metadata()
    if not metadata_entries:
        print("No metadata entries available from the latest run.")
        return []

    # Construct query and get embedding
    query = f"{content_type} {urs_name} {section_name}"
    print(f"Query: {query}")
    query_embedding = sentence_model.encode([query], convert_to_numpy=True)

    # Search FAISS index
    distances, indices = faiss_index.search(query_embedding, top_k)
    print(f"Distances: {distances}, Indices: {indices}, Number of metadata entries: {len(metadata_entries)}")

    # Filter valid indices and retrieve corresponding metadata
    results = [metadata_entries[idx] for idx in indices[0] if idx >= 0 and idx < len(metadata_entries)]
    print(f"Results: {results}")
    return results

# FastAPI app
app = FastAPI()

@app.get("/search")
async def search_documents(urs_name: str, section_name: str, content_type: str, top_k: int = 5):
    """Search FAISS index by urs_name, section_name, and content_type"""
    results = query_faiss_index(urs_name, section_name, content_type, top_k)
    return {"results": results}

# Cleanup database connection on shutdown (optional)
@app.on_event("shutdown")
def cleanup():
    global db_conn, db_cursor
    if db_conn is not None:
        db_cursor.close()
        db_conn.close()
        print("Database connection closed")

TimeoutException: Requesting secret DB_NAME timed out. Secrets can only be fetched when running from the Colab UI.

Run UVICORN server and access via NGROK Tunnel

In [ ]:

import uvicorn
from pyngrok import ngrok
import nest_asyncio

# Mount Google Drive
from google.colab import drive,userdata
drive.mount('/content/drive')

def run_fastapi_with_ngrok():
    """Run FastAPI app and expose via ngrok"""
    nest_asyncio.apply()  # Allow running uvicorn in Colab
    ngrok.set_auth_token(userdata.get('ngrok_auth_token'))  # Replace with your ngrok auth token
    public_url = ngrok.connect(8000)
    print(f"Public URL: {public_url}")
    uvicorn.run(app, host="0.0.0.0", port=8000)
run_fastapi_with_ngrok()

In [ ]:
!python --version

In [ ]:
!curl ifconfig.me